In [14]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
import mysql.connector
from datetime import datetime

In [15]:
# Inisiasi Ollama model
llm = ChatOllama(model="llama3.2", temperature=0)

# List to store message history
chat_history = []

# Set initial system message
system_message = SystemMessage(content="You are a math genius")
chat_history.append(system_message)

In [19]:
# Konfigurasi MySQL
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',  # Kosongkan jika tidak ada password
    'database': 'learn_langchain'
}

In [20]:
# Fungsi untuk menyimpan percakapan ke MySQL
def save_to_database(user_message, ai_response):
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()
        query = """
        INSERT INTO conversations (user_message, ai_response, timestamp)
        VALUES (%s, %s, %s)
        """
        values = (user_message, ai_response, datetime.now())
        cursor.execute(query, values)
        conn.commit()
        cursor.close()
        conn.close()
        print("Conversation saved to database.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

In [21]:
# Chat Loop
def start_chat():
    print("Welcome to the chatbot! Type 'exit' to quit.")
    while True:
        # Get input from User
        query = input("Zuma: ")
        
        # Condition to break the loop
        if query.lower() in ["exit", "bye"]:
            print("AI: Good Bye :(")
            break
        
        # Add the user's query to chat history
        chat_history.append(HumanMessage(content=query))
        
        # Get LLM response
        result = llm(chat_history)
        response = result.content
        
        # Save the LLM response to chat history
        chat_history.append(AIMessage(content=response))
        
        print(f"Human: {query}")
        print(f"AI: {response}")
        
        # Save to MySQL database
        save_to_database(query, response)

In [22]:
# Main Program
if __name__ == "__main__":
    start_chat()

    print("== Message History ==")
    for message in chat_history:
        print(f"{message.role}: {message.content}")

Welcome to the chatbot! Type 'exit' to quit.


: 